<a href="https://colab.research.google.com/github/5quark/Anti-k_t-jet-algorithm/blob/main/Anti_Kt_jet_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import sqrt
import numpy as np
import pandas as pd

$d_{ij} =min(\frac{1}{k^2_{T,i}}, \frac{1}{k^2_{T,j}})\frac{𝚫^2_{ij}}{R^2}  $, i and j for the indices of two particles. 
where
$\Delta_{i j}^{2}=\left(\eta_{i}-\eta_{j}\right)^{2}+\left(\phi_{i}-\phi_{j}\right)^{2}
$, 
$\eta$ is the pseudorapidity and $\phi$ is the azimuth angle.
$d_{iB}=\frac{1}{k^2_{T,i}}$,   B refers to beam axis.

In the limit where the particle is travelling close to the speed of light,
$\eta≈y=\frac{1}{2}\ln(\frac{E+P_T}{E-P_T})$ 
Where
$P_T$: Transverse momentum. This case I took it as $p_z$


In [ ]:
#This cell is for taking the data from the file. 
data=pd.read_csv("#FILE NAME",sep=" ",header=None)
k_t=[]
phi=[]
eta=[]
p_x=[]
p_y=[]
p_z=[]
E=[]
jets=[]
n=20
for i in range(n):
    k_t.append(sqrt(data.iloc[i][0]**2+data.iloc[i][1]**2))
    phi.append(np.arctan(data.iloc[i][1]/data.iloc[i][0]))
    eta.append(np.log((data.iloc[i][2] + data.iloc[i][3])/(data.iloc[i][3]-data.iloc[i][2]))/2)  
    p_x.append(data.iloc[i, 0])
    p_y.append(data.iloc[i, 1])
    p_z.append(data.iloc[i, 2])
    E.append(data.iloc[i, 3])
          
k_t=np.array(k_t)
k_t2=k_t**2
phi=np.array(phi)
eta=np.array(eta)
p_x=np.array(p_x)
p_y=np.array(p_y)
p_z=np.array(p_z)
E=np.array(E)
d_iB=1/k_t**2
jets=np.array(jets)

In [ ]:
#function that I will run in each iteration. 
def func(p_x,p_y,p_z,E,jets):
    n=len(p_x)
    #create k_t, phi and eta arrays
    k_t=[]
    phi=[]
    eta=[]
    k_t=np.array(k_t)
    
    phi=np.array(phi)
    eta=np.array(eta)
    for i in range(n):
        k_t=np.append(k_t,sqrt(p_x[i]**2+p_y[i]**2))
        phi=np.append(phi,np.arctan(p_y[i] / p_x[i]))
        eta=np.append(eta,np.log((p_z[i] + E[i])/( E[i]-p_z[i]))/2) 
    

    #create d_ij and d_iB
    k_t2=k_t**2
    d_iB=1/k_t**2
    d_ij=np.zeros((n,n))
    R=1.0 #I set the distance paramater to 1.0
    for i in range(n):
        for j in range(n):
            delta=(eta[i]-eta[j])**2 + (phi[i]-phi[j])**2
            d_ij[i][j]= np.minimum(k_t2[i],k_t2[j])*delta**2/R**2


    #find the minimum of the d_ij
    min_d_ij=100 #random number
    for i in range(n):
        for j in range(n):
            if(i != j and d_ij[i][j]<min_d_ij): 
                        min_d_ij=d_ij[i][j]         
                        row=i
                        column=j


    if( d_iB[row]>d_ij[row][column] or d_iB[column]>d_ij[row][column])  :   #combine i^th and j^th particles.
        min=min_d_ij
        new_px=p_x[row]+p_x[column]
        new_py=p_y[row]+p_y[column]
        new_pz=p_z[row]+p_z[column]
        new_E=E[row]+E[column]
        p_x=np.append(p_x,new_px)
        p_y=np.append(p_y,new_py)
        p_z=np.append(p_z,new_pz)
        E=np.append(E,new_E)
        p_x=np.delete(p_x,[row,column])
        p_y=np.delete(p_y,[row,column])
        p_z=np.delete(p_z,[row,column])
        E=np.delete(E,[row,column])

    else:                      #declare ith particle as a jet and remove it from the list of particles. 
        jets=np.append(jets,( p_x[np.argmin(d_iB)], p_y[np.argmin(d_iB)],p_z[np.argmin(d_iB)],E[np.argmin(d_iB)] ))
        p_x=np.delete(p_x,[np.argmin(d_iB)])
        p_y=np.delete(p_y,[np.argmin(d_iB)])
        p_z=np.delete(p_z,[np.argmin(d_iB)])
        E=np.delete(E,[np.argmin(d_iB)])

    return p_x,p_y,p_z,E,jets
        

In [ ]:
#run the function until encounter an error, this case having less than 2 elements in four vectors
while True:
    try:
        p_x,p_y,p_z,E,jets=func(p_x,p_y,p_z,E,jets)
    except:
         break
jets=np.append(jets,(p_x,p_y,p_z,E)) # In the p_x,p_y,p_z and E arrays there were only one element in each. I am adding them to the jets array.
print(jets)